In [ ]:
import mne
import os

# Directorio donde se encuentran los archivos EEG
directorio_datos = r"C:\Users\Usuario\Documents\Datos EEG\PWI_DATOS_EEG_PROCESADOS_BUENOS\Set_Average_por_Sujeto_1000ms"

# Diccionario para almacenar los registros EEG por condición
registros_eeg = {"congruente": {}, "incongruente_rel": {}, "incongruente_no_rel": {}}

# Condiciones y sus sufijos de archivo correspondientes
condiciones = {
    "congruente": "PWI_CON",
    "incongruente_rel": "PWI_INC_REL",
    "incongruente_no_rel": "PWI_INC_UNREL"
}

# Sujetos a excluir de la carga de archivos, pero deben aparecer en el diccionario
sujetos_excluir = {11, 30}

# Iterar sobre los 40 sujetos para cada condición
for sujeto_id in range(1, 41):
    for condicion, sufijo_archivo in condiciones.items():
        sujeto_key = f"Sujeto_{sujeto_id}"

        if sujeto_id in sujetos_excluir:
            registros_eeg[condicion][sujeto_key] = None
            print(f"{sujeto_key} excluido de la carga en '{condicion}', asignado como None.")
            continue  # Saltar a la siguiente condición

        nombre_archivo = f"S{sujeto_id}_{sufijo_archivo}.set"
        ruta_archivo = os.path.join(directorio_datos, nombre_archivo)

        try:
            # Cargar el archivo .set como un registro continuo
            registro_continuo = mne.io.read_raw_eeglab(ruta_archivo)
            registros_eeg[condicion][sujeto_key] = registro_continuo
            print(f"{nombre_archivo} cargado exitosamente en '{condicion}'.")
        except FileNotFoundError:
            registros_eeg[condicion][sujeto_key] = None
            print(f"{nombre_archivo} no encontrado en '{condicion}'.")
        except Exception as e:
            registros_eeg[condicion][sujeto_key] = None
            print(f"Error al cargar {nombre_archivo} en '{condicion}': {e}")

# Resumen de la carga de archivos
print("\nResumen de las cargas:")
for condicion, sujetos in registros_eeg.items():
    print(f"\nCondición: {condicion}")
    for sujeto, datos in sujetos.items():
        estado = "Cargado" if datos is not None else "No cargado"
        print(f"  {sujeto}: {estado}")


In [ ]:
import numpy as np
import mne

def escalar_minmax_personalizado(datos, min_abs, max_abs):
    # Escalar valores entre -1 y 1 usando el máximo absoluto
    return datos / max_abs

def calcular_minmax_scaler_registros_eeg(registros_eeg):
    
    registros_eeg_minmax = {"congruente": {}, "incongruente_rel": {}, "incongruente_no_rel": {}}

    for sujeto in registros_eeg["congruente"]:  # Iterar sobre sujetos
        if registros_eeg["congruente"][sujeto] is None:
            # Si el sujeto fue excluido, mantenerlo como None en todos los casos
            for cond in registros_eeg_minmax:
                registros_eeg_minmax[cond][sujeto] = None
            continue

        # Obtener datos de la condición congruente
        datos_congruente = registros_eeg["congruente"][sujeto].get_data()  # (Canales x Tiempos)

        # Calcular máximo absoluto por canal para escalar entre -1 y 1
        max_abs_congruente = np.max(np.abs(datos_congruente), axis=1, keepdims=True)
        max_abs_congruente[max_abs_congruente == 0] = 1  # Evitar divisiones por cero

        # Normalizar todas las condiciones respecto a la condición congruente
        for condicion in registros_eeg:
            if registros_eeg[condicion][sujeto] is not None:
                datos_condicion = registros_eeg[condicion][sujeto].get_data()
                datos_escalados = datos_condicion / max_abs_congruente

                # Crear nueva instancia de RawArray con los datos normalizados
                info = registros_eeg[condicion][sujeto].info  # Mantener la metadata original
                registros_eeg_minmax[condicion][sujeto] = mne.io.RawArray(datos_escalados, info)
            else:
                registros_eeg_minmax[condicion][sujeto] = None  # Mantener sujetos sin datos

    return registros_eeg_minmax

# Aplicar el escalado a los datos cargados
registros_eeg_minmax = calcular_minmax_scaler_registros_eeg(registros_eeg)
